DMD FUNCTIONS - FAT matrix X - for ONLINE DMD comparison - Zhang, Rowley,...

In [2]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from scipy import linalg

In [ ]:
def fGEDMD(X, Y, tol=1e-12, type: str=None, k=-1, weights=None): 
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=X.shape[1]; N=X.shape[0]
    D = np.linalg.norm(X, axis=0)
    X = X/D; Y=Y/D
    if(weights is not None):
        X = X*weights; Y=Y*weights
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) 
    V=np.conjugate(V.T)
    if(k==-1):
        k=m
        for i in range(1,min(N,m)):
            if(Sigma[i]<=Sigma[0]*tol):
                k=i
                break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    A = Y @ (V/Sigma) @ np.conjugate(U.T)
    """S_k = np.conjugate(U.T)@B_k ###nema potrebe za ovim! - direktno eig..
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W"""
    if(type=="exact"):
        return "nema jos exact implementacije..."
        Z = B_k@W
    Lambda, Z = np.linalg.eig(A)
    #print(A @ Z - ((np.multiply(Z, Lambda))))
    r = np.linalg.norm(A @ Z - np.multiply(Z, Lambda), axis=0)
    
    return Z, Lambda, r

In [ ]:
def fGEDMDQ_multiple_trajectories(X, Y, tol=1e-12, type: str=None):
    m = X.shape[0]
    M = np.bmat([X.T, Y.T])
    Q, L = np.linalg.qr(M)
    #print(Q[:,:m].T@Q[:,:m]) #provjera ortogonalnosti
    L = L.T
    Z, Lambda, r = fGEDMD(L[:m,:m], L[m:,:m], tol, type)
    return Z, Lambda, r, Q, L 

In [ ]:
def fDMD_added_snap(v, Q, L, tol=1e-12, type: str=None, k=-1, weights=None, ngram=5):
    #v tipa 1 x 2m => prvih m je zadnji element u Y, drugih m je novi element
    # ngram - treba li tu reortogonalizacija?
    m=Q.shape[1]//2; n=Q.shape[0]
    R = np.bmat([[L.T],[v.T]])
    U, T = np.linalg.qr(R)
    T=T.T #novi L - sad je T donjetrokutasto
    Q = np.bmat([[Q@U[:2*m,:]],[U[-1,:]]])
    #novi Z i Lambda i r
    Z, Lambda, r = fGEDMD(T[:m,:m], T[m:,:m], tol, type)
    return Z, Lambda, r, Q, T


In [ ]:
### Ovo bi trebalo biti isto?

def DMD_alpha_for_reconstruction(X, Z, indices, L, weights=None):
    """X = snapshotovi - prvih m (bez m+1-vog) - dakle X, a ne S
    Z = modes,
    indices = indices which we want to work with - from 1/r graph
    weights = np.array tip ili lista"""
    #treba li se formirati Vandermondeova matrica?
    if (indices == 'all'):
        indices=np.array([i for i in range (Z.shape[1])])
    m=X.shape[1]; l=indices.shape[0]
    Z = Z[:,indices]
    Q, R = np.linalg.qr(Z) # Q je tipa duljina_snapshota(N) x l, R je tipa lxl
    if(weights is None):
        weights=np.ones((m)).reshape(-1)
    weights=np.array(weights)
    pom=np.vander(L[indices], m, increasing=True)*weights
    alpha= np.multiply(np.conj(R.T)@R, np.conj(pom @ np.conj(pom.T)))
    G = (np.conj(Q.T) @ X)[:l, :]  
    alpha = scipy.linalg.solve(alpha, np.multiply(np.conj(pom),(np.conj(R.T)@G))@np.ones((m, 1)), assume_a='pos')
    #alpha = scipy.linalg.solve(alpha, np.multiply(np.conj(pom),(np.conj(R.T)@G*weights))@np.ones((m, 1)), assume_a='pos')
    return Z, L[indices], alpha.reshape(-1)

def DMD_reconstruction(X, Z, indices, L, times, weights=None, real=True): #mozda da prima vektor napraviti..
    """X = snapshotovi - prvih m (bez m+1-vog) - dakle X, a ne S,
    Z = dmd modes (returned from some version of DMD, ex. GEDMDQ)
    L = dmd eigs (returned from some version of DMD, ex. GEDMDQ)
    time = integer, which datasnapshot you want to reconstruct/predict"""
    Z_l, L, alpha = DMD_alpha_for_reconstruction(X, Z, indices, L, weights)
    num=np.asarray(times).shape[0]
    recs = np.empty((Z_l.shape[0], num), dtype='complex_')
    for i in range(num):
        recs[:,i] = Z_l@(L**(times[i])*alpha) #mislim da je times[i] jer ovdje krecemo od 0, za razliku od matlaba gdje krecemo od 1
    if real:
        return np.real(recs) #ako sve realno, ovo ce biti realno za svaki i, samo ce zapis biti u obliku kompleksnog; zato saljemo np.real(recs)!
    else:
        return recs